In [1]:
from pyspark.sql import SparkSession, functions as f

In [2]:
spark = (
    SparkSession.builder.appName("Preprocessing_Yellow")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '3g')   
    .config('spark.executor.memory', '4g')  
    .config('spark.executor.instances', '2')  
    .config('spark.executor.cores', '2')
    .getOrCreate()
) 

your 131072x1 screen size is bogus. expect trouble
23/09/25 15:34:48 WARN Utils: Your hostname, DESKTOP-SATV84A resolves to a loopback address: 127.0.1.1; using 172.26.254.29 instead (on interface eth0)
23/09/25 15:34:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/25 15:34:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
descriptions = spark.read.parquet("../../../data/insights/pre_insights/descriptions.parquet/")

In [4]:
tags_count = descriptions.groupBy("merchant_abn").agg(f.countDistinct("merchant_description").alias("description_count"))
print(tags_count.filter(tags_count.description_count > 1).count())

0


In [5]:
descriptions_agg = descriptions.groupBy("merchant_abn").\
            agg(f.first("merchant_description").alias("merchant_description"))

In [6]:
descriptions_count = descriptions_agg.groupBy("merchant_description").agg(f.count("merchant_abn").alias("number_of_merchants_with_description"))

In [7]:
descriptions_count.orderBy("number_of_merchants_with_description")

merchant_description,number_of_merchants_with_description
"jewelry, watch, c...",91
art dealers and g...,112
telecom,125
antique shops - s...,129
"equipment, tool, ...",134
"hobby, toy and ga...",142
"opticians, optica...",151
motor vehicle sup...,151
lawn and garden s...,153
"stationery, offic...",161


In [8]:
tech_and_electronics = ["computer", "digital", "television", "telecom"]
retail_and_novelty = ["newspaper", "novelty", "hobby", "shoe", "instruments", "bicycle", "craft","office"]
garden_and_furnishings = ["florists", "furniture", "garden", "tent"]
antiques_and_jewellery = ["galleries", "antique", "jewelry"]
specialized_services = ["health", "motor", "opticians"]

In [9]:
def segment(description):
    for segment, keywords in [("tech_and_electronics", tech_and_electronics),
                               ("retail_and_novelty", retail_and_novelty),
                               ("garden_and_furnishings", garden_and_furnishings),
                               ("antiques_and_jewellery", antiques_and_jewellery),
                               ("specialized_services", specialized_services)]:
        if any(keyword in description for keyword in keywords):
            return segment
    return "other"

In [10]:
from pyspark.sql.types import StringType
segment_udf = f.udf(segment, StringType())

In [11]:
descriptions_agg = descriptions_agg.withColumn("segment", segment_udf(descriptions_agg.merchant_description))

In [12]:
descriptions_agg.write.mode("overwrite").parquet("../../../data/insights/agg_insight_data/descriptions_agg.parquet")

In [13]:
spark.stop()

23/09/25 15:35:03 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1107)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:244)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:2088)
	at org.apache.spark.Heartbeater$$anon$1.run(Heartbeater.scala:46)
	at java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:511)
	at java.util.concurrent.FutureTask.runAndReset(FutureTask.java:308)
	at java.util.concurrent.ScheduledThreadPoolExecutor$ScheduledFutureTask.access$301(ScheduledThreadPoolExecutor.java:1

### Insights
- There 25 different descriptions for all merchants